<a href="https://colab.research.google.com/github/dongchanlim/Blackjack/blob/master/New_blackjack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import pandas as pd
import numpy as np

# Initial entire Card Deck (블랙잭 카드 구성하기)
class Card:
  def __init__(self):
    self.symbols = [i for i in range(2,11)] * 4 + ['A','J','Q','K'] * 4
    self.numbers = [i for i in range(2,11)] * 4 + [11, 10, 10, 10] * 4
    self.deck = dict(zip(self.symbols, self.numbers))

In [0]:
class Pack:
  def __init__(self):
    self.initial = random.choices(list(Card().deck.items()), k = 2)
    # Player().initial[0][0] - Symbol
    # Player().initial[0][1] - Number 
    self.symbol = [self.initial[i][0] for i in range(len(self.initial))]
    self.number = [self.initial[i][1] for i in range(len(self.initial))]
    self.total = sum(self.number)
    # 카드 숫자 세기
    self.card_number = 2
    # 블랙잭인지 여부
    self.blackjack = False
    # 21인지 여부
    self.twentyone = False
    # 21 넘었는지 여부
    self.over_21 = False
    self.a_exist = False
    
  def add_card(self):
    self.initial += random.choices(list(Card().deck.items()), k = 1)
    self.symbol = [self.initial[i][0] for i in range(len(self.initial))]
    self.card_number += 1

  def add_card_effect(self):
    self.number = [self.initial[i][1] for i in range(len(self.initial))]
    self.total = sum(self.number)
    if 'A' in self.symbol:
      self.a_exist = True
    

In [0]:
class Player:
  def __init__(self, name, money):
    self.pack = Pack()
    # 승/패 체크
    self.lose = False
    # 플레이어 이름 정하기
    self.name = name
    # 플레이어 자금 정하기
    self.money = money
    # 플레이어 배팅 정하기
    self.betting = 0

  def set_name(self):
    self.name = input("Who is going to play: ")
  def set_money(self):
    self.money = float(input("Hi. {}! How much do you want to start with?($): ".format(self.name)))
    print()
  def set_betting(self):
    self.betting = float(input("How much are you going to bet? "))
    while self.betting > self.money:
      # 배팅머니가 가지고있는 돈을 넘을경우 에러메세지
      print("Betting money exceed the money you owned\n")
      self.betting = float(input("How much are you going to bet? "))
  
  # 히트(한장 더 받기)
  def hit(self):
    self.pack.add_card()

  def select_a_initial(self):
    if 'A' in self.pack.symbol[:2]:
      if self.pack.total > 21:
        self.pack.number[self.pack.symbol.index("A")] = 1
        # self.pack.total -= 10
      else:
        a_number = int(input("{}, Which number do you want for Ace?(1/11): ".format(self.name)))
        if a_number == 1:
          self.pack.number[self.pack.symbol.index("A")] = 1
          # self.pack.total -= 10
        elif a_number == 11:
          self.pack.number[self.pack.symbol.index("A")] = 11
        else:
          print("Invalid number. Please select either 1 or 11.")
          a_number = int(input("{}, Which number do you want for Ace?(1/11): ".format(self.name)))

  def select_a_afterhit(self):
    if 'A' in self.pack.symbol[2:]:
      if self.pack.total > 21:
        self.pack.number[self.pack.symbol.index("A")] = 1
        # self.pack.total -= 10
      else:
        a_number = int(input("{}, Which number do you want for Ace?(1/11): ".format(self.name)))
        if a_number == 1:
          self.pack.number[self.pack.symbol.index("A")] = 1
          # self.pack.total -= 10
        elif a_number == 11:
          self.pack.number[self.pack.symbol.index("A")] = 11
        else:
          print("Invalid number. Please select either 1 or 11.")
          a_number = int(input("{}, Which number do you want for Ace?(1/11): ".format(self.name)))

In [0]:
class Dealer:
  def __init__(self):
    self.pack = Pack()
    self.lose = False
    self.name = 'Dealer'

  def hit(self):
    self.pack.add_card()

  def select_a(self):
    if 'A' in self.pack.symbol:
      if self.pack.total <= 17:
          self.pack.number[self.pack.symbol.index("A")] = 1
          # self.pack.total -= 10
          self.pack.a_exist = True

In [0]:
class Blackjack:
  def __init__(self, play_number, name, money):
    # 몇번째 게임인지 보여주기
    self.play_number = play_number
    # 플레이어와 딜러 오브젝트 만들기
    self.player = Player(name, money)
    self.dealer = Dealer()
    # 게임을 멈출지 안멈출지 정하기
    self.game_stop = False

  # 처음 상황 세팅하기
  def initial_setting(self):
    self.player.set_name()
    self.player.set_money()

  def replay(self):
    if self.player.money == 0:
      self.game_stop = True
    else:
      self.player.lose = False
      self.dealer.lose = False
      self.play_number += 1
  
  # 플레이어의 현재 플레이 머니를 보여준다
  def display_money_status(self):
    print("{}'s Play Money : ${}".format(self.player.name, self.player.money))

  # 플레이어 또는 딜러 (target) 의 현재 카드 상황을 보여준다
  def display_current_card(self, target):
    if target == self.player:
      print("{}: {}, sum: {}".format(target.name, target.pack.symbol, target.pack.total))
    # 딜러의 카드 하나는 뒤집은 상태에서 나머지 카드들만 보여준다
    elif target == self.dealer:
      print("Dealer: {}, sum: ?".format(target.pack.symbol[:-1] + ["?"]))

  
  # 블랙잭 (처음 2장이 21) 인지 확인
  def check_blackjack(self, player, dealer):
    if player.pack.card_number == 2:
      if player.pack.total == 21:
        player.pack.blackjack = True
        dealer.lose = True
        self.game_stop = True
        
      elif dealer.pack.total == 21:
        dealer.pack.blackjack = True
        player.lose = True
        self.game_stop = True
        
  
  # 3장 부터 21 인지 확인 
  def check_twentyone(self, player, dealer):
    if player.pack.card_number > 2:
      if player.pack.total == 21:
        player.pack.twentyone = True
        dealer.lose = True
        self.game_stop = True
      elif dealer.pack.total == 21:
        dealer.pack.twentyone = True
        player.lose = True
        self.game_stop = True

  def over_21(self, player, dealer):
    if player.pack.total > 21:
      player.lose = True
      self.game_stop = True
    elif dealer.pack.total > 21:
      dealer.lose = True
      self.game_stop = True

  def a_exist_display(self):
    if self.player.pack.a_exist:
      print("")
      self.display_current_card(self.player)
      self.display_current_card(self.dealer)

  def who_win(self, player, dealer):
    if not dealer.lose and not player.lose:
      if dealer.pack.total > player.pack.total:
        player.lose = True
      elif dealer.pack.total < player.pack.total:
        dealer.lose = True
      else:
        pass
      self.game_stop = True
    
   

In [0]:
class Game:
  def __init__(self, name, money):
    self.play_number = 0
    self.game = Blackjack(self.play_number, name, money)
    self.hit = ""
  
  # 단판 게임
  def first_setting(self):
    print("Welcome to Blackjack.\n")
    self.game.initial_setting()
    print("\n")
    self.game.display_money_status()

  # hit or stop 정하기
  def hit_stop(self):
    self.hit = input("Hit / Stop ? (H/S): ")
    if self.hit.upper() == "H":
      self.game.player.hit()
    elif self.hit.upper() == "S":
      self.game.game_stop = True
    else:
      print("Invalid Command. Please type only H or S")
      self.hit = input("Hit / Stop ? (H/S): ")

  def after_hit(self):
    self.game.player.select_a_afterhit()
    self.game.dealer.select_a()
    self.game.a_exist_display()
    for i in [self.game.player, self.game.dealer]:
      i.pack.add_card_effect()
    print("\n-Table-")
    for i in [self.game.player, self.game.dealer]:
      self.game.display_current_card(i)
    self.game.check_twentyone(self.game.player, self.game.dealer)
    self.game.over_21(self.game.player, self.game.dealer)

  def play(self):
    self.game.replay()
    while not self.game.game_stop:
      self.game.player.set_betting()
      print("\n-Table-")
      print("Play number {}.\n".format(self.game.play_number))
      # 처음 카드상황을 보여준다
      for i in [self.game.player, self.game.dealer]:
        self.game.display_current_card(i)
      # 플레이어와 딜러의 카드가 블랙잭인지 확인한다
      self.game.check_blackjack(self.game.player, self.game.dealer)
      self.game.player.select_a_initial()
      self.game.dealer.select_a()
      self.game.a_exist_display()
      while not self.game.game_stop and self.hit.upper() != "S":
        self.hit_stop()
        self.after_hit()
      while self.hit.upper() == "S" and self.game.dealer.pack.total < 17:
      # 플레이어가 스탑을 선택하고 딜러의 카드의 합계가 17 이상이 될때까지 카드를 추가하기
        print("\nAdding one more card for dealer")
        self.game.dealer.hit()
        self.after_hit()

      self.game.who_win(self.game.player, self.game.dealer)
    # 돈을 땃는지 잃엇는지 결과에 적용
    self.betting_result()
  
  def display_final_card(self):
    # 마지막 카드 상황 보여주기
    print("\n-Final Result-")
    for i in [self.game.player, self.game.dealer]:
      print("{}: {}, sum: {}".format(i.name, i.pack.symbol, i.pack.total))

  def betting_result(self):
    if self.game.player.lose:
      self.game.player.money -= self.game.player.betting
    elif self.game.dealer.lose:
      self.game.player.money += self.game.player.betting

  def display_final(self):
    self.display_final_card()
    print("")
    if self.game.player.pack.blackjack:
      print("Blackjack! Player Win!")
    elif self.game.dealer.pack.blackjack:
      print("Blackjack! Dealer Win!")
    elif self.game.player.pack.twentyone:
      print("21! Player Win!")
    elif self.game.player.pack.twentyone:
      print("21! Dealer Win")
    elif self.game.player.pack.total > 21:
      print("Since number is over 21\nDealer win!")
    elif self.game.dealer.pack.total > 21:
      print("Since number is over 21\nPlayer win!")
    # 21 / 블랙잭 / 버스트 가 아닐때 카드 합계를 비교하여 승패 정하기
    elif self.game.player.lose:
      print("Dealer Win!")
    elif self.game.dealer.lose:
      print("Player Win!")
    else:
      print("Draw")
    print("")
    self.game.display_money_status()

    

In [0]:
def main():
  a = Game("", 0)
  a.first_setting()
  a.play()
  a.display_final()
  again = input("Do you want to play agian?: (y/n): ")
  while a.game.player.money != 0 and again.upper() != "N":
    if again.upper() == "Y":
      b = Game(a.game.player.name, a.game.player.money)
      b.play()
      b.display_final()
      a.game.player.name, a.game.player.money = b.game.player.name, b.game.player.money
      again = input("Do you want to play agian?: (y/n): ")
    else: 
      print("Invalid Answer. Please type correctly\n")
      again = input("Do you want to play agian?: (y/n): ")
  print("")
  if a.game.player.money == 0:
    print("You are bankrupted. Can't play.")
  print("Game over")

In [95]:
if __name__ == '__main__':
    main()

Welcome to Blackjack.

Who is going to play: dong
Hi. dong! How much do you want to start with?($): 5000



dong's Play Money : $5000.0
How much are you going to bet? 500

-Table-
Play number 1.

dong: [9, 'Q'], sum: 19
Dealer: [5, '?'], sum: ?
Hit / Stop ? (H/S): s

-Table-
dong: [9, 'Q'], sum: 19
Dealer: [5, '?'], sum: ?

Adding one more card for dealer

-Table-
dong: [9, 'Q'], sum: 19
Dealer: [5, 6, '?'], sum: ?

-Final Result-
dong: [9, 'Q'], sum: 19
Dealer: [5, 6, 'K'], sum: 21

Dealer Win!

dong's Play Money : $4500.0
Do you want to play agian?: (y/n): y
How much are you going to bet? 500

-Table-
Play number 1.

dong: [8, 3], sum: 11
Dealer: [3, '?'], sum: ?
Hit / Stop ? (H/S): h

-Table-
dong: [8, 3, 4], sum: 15
Dealer: [3, '?'], sum: ?
Hit / Stop ? (H/S): h

-Table-
dong: [8, 3, 4, 7], sum: 22
Dealer: [3, '?'], sum: ?

-Final Result-
dong: [8, 3, 4, 7], sum: 22
Dealer: [3, 2], sum: 5

Since number is over 21
Dealer win!

dong's Play Money : $4000.0
Do you want to play agian?: 